<a href="https://colab.research.google.com/github/liangchow/zindi-amazon-secret-runway/blob/explore-sentinel1-data/Generate_airstrip_masks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports and Setup.

In [56]:
%%capture
!pip -q install rasterio

In [57]:
import rasterio
import geopandas as gpd
from rasterio.features import rasterize
import numpy as np
import os

### Mount your Google Drive and install project files

First, we'll mount your Google Drive. Then we'll clone the main branch from the GitHub repo so we have access to all of the files from there.

In [58]:
# mount your drive in case you have any new data uploaded there you want to use
from google.colab import drive
drive.mount('/content/drive')

# clone the main branch from GitHub to get all the data and files from there onto the current runtime session
!apt-get install git
!git clone https://github.com/liangchow/zindi-amazon-secret-runway.git
!git pull # pulls the latest changes from repo

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
fatal: destination path 'zindi-amazon-secret-runway' already exists and is not an empty directory.
fatal: not a git repository (or any of the parent directories): .git


**Note: Before you run the following cell**

The training images are store in a shared folder `Sentinel` on Google Drive. To access the shared folder in Google Colab, first create a shortcut to the shared folder. Copy the shortcut to your Google Drive.

Now you can access the folder `Sentinel` like any other folder in your Google Drive.

In [59]:
shared_drive_path = "/content/drive/MyDrive/Sentinel"

# List files in the shared drive
# print(os.listdir(shared_drive_path))

In [60]:
# Load the polygon shapefile
airstrips_gdf = gpd.read_file("/content/zindi-amazon-secret-runway/Data_Visualization/data/pac_2024_training/pac_2024_training.shp")

In [68]:
# Load one of the multiband rasters to get the crs
with rasterio.open(shared_drive_path + "/training/images/Sentinel_AllBands_Training_Id_1.tif") as src:
    crs = src.crs

In [62]:
# Convert the GeoDataFrame to match the raster CRS
airstrips_projected_gdf = airstrips_gdf.to_crs(crs)

Create a buffer around each of the airstrip

In [64]:
# Define buffer distance
buffer_distance = 200

# Buffer the airstrips
polygon_gdf['geometry'] = airstrips_projected_gdf.geometry.buffer(buffer_distance)

In [65]:
# Use a subset of the airstrips for testing
polygon_gdf_subset = polygon_gdf.head(2)

To process all airstrip use polygon_gdf instead of polygon_gdf_subset in the cell below

In [67]:
# Iterate over each polygon to create individual mask rasters
#for idx, row in polygon_gdf.iterrows():
for idx, row in polygon_gdf_subset.iterrows():
    # Get the airstrip ID
    airstrip_id = row['id']

    # Check for an associated TIFF file
    img = f"{shared_drive_path}/training/images/Sentinel_AllBands_Training_Id_{airstrip_id}.tif"

    if not os.path.exists(img):
        print(f"No TIFF file found for airstrip ID: {airstrip_id}")
    else:
        print(f"Creating mask for airstrip ID: {airstrip_id}")
        # Initialize a blank mask for the current polygon
        mask = np.zeros((height, width), dtype='uint8')

        # Load the TIFF file and read it
        with rasterio.open(img) as img_src:
            # Get the raster metadata (to match extent and resolution)
            img_meta = img_src.meta.copy()
            # Create an affine transformation
            img_transform = img_src.transform
            # Get raster dimensions
            img_width = img_src.width
            img_height = img_src.height

            # Set mask to 1 where the airstrip buffer is present
            shape = [(row.geometry, 1)]
            mask = rasterize(
                shapes=shape,
                out_shape=(img_height, img_width),
                transform=img_transform,
                fill=0,   # Assign 0 to areas outside the polygon
                dtype='uint8'
            )

            # Update metadata for the output raster
            out_meta = img_meta.copy()
            out_meta.update({
                "count": 1,
                "dtype": "uint8",
                "nodata": None  # Disable nodata (otherwise QGIS will not display the 0s)
            })

            # Save the individual mask raster
            output_raster = f"Mask_Airstrip_Id_{airstrip_id}.tif"
            #output_raster = f"{shared_drive_path}/training/masks/Mask_Airstrip_Id_{airstrip_id}.tif"
            with rasterio.open(output_raster, "w", **out_meta) as dest:
                dest.write(mask, 1)

            print(f"Saved mask for airstrip ID {airstrip_id} as {output_raster}")

Creating mask for airstrip ID: 1
Saved mask for airstrip ID 1 as Mask_Airstrip_Id_1.tif
Creating mask for airstrip ID: 2
Saved mask for airstrip ID 2 as Mask_Airstrip_Id_2.tif
